## Back Test Results Binary Classification

In [1]:
import pandas as pd
from datetime import datetime, timezone
from pycaret.datasets import get_data
from pycaret.classification import *
import yaml
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    pair = cfg['currency']['pair']
    frac = cfg['model']['frac']
    random_state = cfg['model']['random_state']

In [3]:
print('Pair:', pair)
print('Fraction:', frac)
print('Random State:', random_state)

Pair: EURUSD
Fraction: 0.7
Random State: 700


In [4]:
now = datetime.now()
today = now.strftime("%d-%m-%Y_%I-%M_%p")

In [5]:
dataset = pd.read_csv('01_Back_Test_Data.csv')
dataset.shape

(4000, 15)

In [6]:
dataset.describe()

,Candle_No,Current_Market_Fit,Current_Market,Rec1_Score,Rec1_HH,Rec1_LL,Rec2_Score,Rec2_HH,Rec2_LL,Rec3_Score,Rec3_HH,Rec3_LL,Rec4_Score,Rec4_HH,Rec4_LL
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,5467.304500,44.307500,0.500500,0.898500,-0.004404,0.004235,0.670250,-0.004236,0.004127,-0.872500,-0.004120,0.004125,1.807250,-0.004234,0.003948
std,2796.602168,30.162226,0.500062,54.079459,0.003299,0.003354,54.989017,0.003144,0.003158,55.000004,0.002991,0.003178,55.279879,0.003147,0.002981
min,0.000000,0.000000,0.000000,-97.000000,-0.028300,0.000000,-99.000000,-0.027400,0.000100,-98.000000,-0.027700,0.000000,-99.000000,-0.027700,0.000000
25%,3194.750000,15.000000,0.000000,-45.000000,-0.005800,0.002000,-45.000000,-0.005600,0.001900,-50.000000,-0.005600,0.001900,-47.000000,-0.005700,0.001900
50%,6391.000000,45.000000,1.000000,0.000000,-0.003600,0.003500,0.000000,-0.003500,0.003400,0.000000,-0.003400,0.003400,0.000000,-0.003400,0.003200
75%,7968.750000,71.000000,1.000000,48.000000,-0.002100,0.005600,50.000000,-0.002000,0.005425,48.000000,-0.002000,0.005500,53.000000,-0.002000,0.005300
max,9637.000000,100.000000,1.000000,100.000000,-0.000000,0.047900,100.000000,-0.000000,0.047300,98.000000,-0.000000,0.047300,98.000000,-0.000000,0.028900


In [8]:
dataset.head(20)

,Candle_No,Current_Market_Fit,Current_Market,Rec1_Score,Rec1_HH,Rec1_LL,Rec2_Score,Rec2_HH,Rec2_LL,Rec3_Score,Rec3_HH,Rec3_LL,Rec4_Score,Rec4_HH,Rec4_LL
0,7684.0,79.0,0.0,15.0,-0.0024,0.0042,-31.0,-0.0029,0.0029,87.0,-0.0069,0.0013,84.0,-0.0049,0.0032
1,5362.0,67.0,1.0,89.0,-0.0111,0.0049,27.0,-0.0068,0.0035,0.0,-0.0040,0.0033,-92.0,-0.0013,0.0108
2,4503.0,61.0,1.0,-38.0,-0.0016,0.0088,0.0,-0.0039,0.0022,-2.0,-0.0011,0.0016,9.0,-0.0057,0.0063
3,6941.0,53.0,1.0,66.0,-0.0055,0.0032,79.0,-0.0094,0.0053,97.0,-0.0116,0.0023,-70.0,-0.0013,0.0072
4,4411.0,85.0,1.0,45.0,-0.0046,0.0026,61.0,-0.0095,0.0103,71.0,-0.0022,0.0030,-46.0,-0.0004,0.0076
5,8542.0,18.0,0.0,28.0,-0.0032,0.0010,-47.0,-0.0003,0.0077,5.0,-0.0011,0.0025,-66.0,-0.0023,0.0063
6,7975.0,25.0,1.0,37.0,-0.0072,0.0017,-85.0,-0.0009,0.0088,45.0,-0.0040,0.0027,77.0,-0.0039,0.0014
7,7971.0,55.0,0.0,37.0,-0.0069,0.0001,-25.0,-0.0030,0.0043,1.0,-0.0069,0.0054,-5.0,-0.0028,0.0058
8,7274.0,88.0,1.0,63.0,-0.0085,0.0029,57.0,-0.0045,0.0020,52.0,-0.0037,0.0015,-71.0,-0.0006,0.0046
9,8621.0,81.0,0.0,-8.0,-0.0010,0.0054,84.0,-0.0082,0.0007,50.0,-0.0049,0.0011,0.0,-0.0027,0.0062


## Drop Predictions with Less than 20% LR Fit

In [9]:
dataset = dataset[dataset['Current_Market_Fit'] > 15]
dataset = dataset.reset_index()
del dataset['index']

In [10]:
dataset.shape

(2985, 15)

In [11]:
dataset = dataset.drop([
    'Candle_No',
    'Current_Market_Fit',
    'Rec1_Score',
    'Rec2_Score',
    'Rec3_Score',
    'Rec4_Score',
], axis=1)

In [12]:
dataset.head()

,Current_Market,Rec1_HH,Rec1_LL,Rec2_HH,Rec2_LL,Rec3_HH,Rec3_LL,Rec4_HH,Rec4_LL
0,0.0,-0.0024,0.0042,-0.0029,0.0029,-0.0069,0.0013,-0.0049,0.0032
1,1.0,-0.0111,0.0049,-0.0068,0.0035,-0.0040,0.0033,-0.0013,0.0108
2,1.0,-0.0016,0.0088,-0.0039,0.0022,-0.0011,0.0016,-0.0057,0.0063
3,1.0,-0.0055,0.0032,-0.0094,0.0053,-0.0116,0.0023,-0.0013,0.0072
4,1.0,-0.0046,0.0026,-0.0095,0.0103,-0.0022,0.0030,-0.0004,0.0076


In [13]:
data = dataset.sample(frac=frac, random_state=random_state).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))
print('Fraction: ' + str(frac))
#print('Random State: ', str(random_state))

Data for Modeling: (2090, 9)
Unseen Data For Predictions: (895, 9)
Fraction: 0.7


# Create Unseen DataFrame

In [14]:
data_unseen.shape

(895, 9)

In [15]:
data_unseen.head()

,Current_Market,Rec1_HH,Rec1_LL,Rec2_HH,Rec2_LL,Rec3_HH,Rec3_LL,Rec4_HH,Rec4_LL
0,0.0,-0.0093,0.0047,-0.0072,0.0213,-0.0021,0.0043,-0.0060,0.0036
1,0.0,-0.0005,0.0074,-0.0065,0.0041,-0.0025,0.0044,-0.0023,0.0044
2,1.0,-0.0017,0.0050,-0.0047,0.0016,-0.0018,0.0047,-0.0013,0.0026
3,1.0,-0.0020,0.0077,-0.0054,0.0031,-0.0079,0.0024,-0.0023,0.0078
4,1.0,-0.0053,0.0007,-0.0017,0.0011,-0.0038,0.0007,-0.0025,0.0044


In [16]:
exp_name = setup(data = data,
                 target = 'Current_Market',
#                 fold_shuffle=True,
                 session_id=123,
                 silent=True,
#                 n_jobs = 64,
#                 imputation_type='iterative'
                ) 

,Description,Value
0,session_id,123
1,Target,Current_Market
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(2090, 9)"
5,Missing Values,False
6,Numeric Features,8
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [17]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.5185,0.5184,0.5207,0.5269,0.5235,0.0369,0.0368,0.1500
lr,Logistic Regression,0.5096,0.4973,1.0000,0.5096,0.6751,0.0000,0.0000,0.2540
ridge,Ridge Classifier,0.5096,0.0000,1.0000,0.5096,0.6751,0.0000,0.0000,0.1400
lda,Linear Discriminant Analysis,0.5069,0.5007,0.6538,0.5116,0.5735,0.0082,0.0097,0.0050
nb,Naive Bayes,0.5042,0.4764,0.7265,0.5090,0.5980,-0.0001,0.0012,0.1530
knn,K Neighbors Classifier,0.5034,0.5147,0.5127,0.5135,0.5128,0.0066,0.0066,0.1710
qda,Quadratic Discriminant Analysis,0.4986,0.4907,0.6646,0.5051,0.5730,-0.0092,-0.0076,0.0060
ada,Ada Boost Classifier,0.4973,0.4922,0.5222,0.5052,0.5129,-0.0064,-0.0064,0.0250
gbc,Gradient Boosting Classifier,0.4966,0.5009,0.5169,0.5060,0.5098,-0.0075,-0.0077,0.0420
svm,SVM - Linear Kernel,0.4952,0.0000,0.3000,0.1524,0.2021,0.0000,0.0000,0.1540


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=123, splitter='best')

/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_s

/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute standard_coef_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute standard_intercept_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute average_coef_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute average_intercept_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/mladmin/Data/Forex

/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/mladmin/Data/Forexience/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_s

In [ ]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    model = cfg['model']['name']

In [ ]:
print('Model Name:', model)

In [ ]:
MODEL = create_model(model)

In [ ]:
plot_model(MODEL, plot = 'confusion_matrix')

In [ ]:
TUNED_MODEL = tune_model(MODEL)

In [ ]:
plot_model(TUNED_MODEL, plot = 'auc')

In [ ]:
plot_model(TUNED_MODEL, plot = 'pr')

In [ ]:
plot_model(TUNED_MODEL, plot='feature')

In [ ]:
plot_model(TUNED_MODEL, plot = 'confusion_matrix')

In [ ]:
predict_model(TUNED_MODEL);

In [ ]:
FINAL_MODEL = finalize_model(TUNED_MODEL)

In [ ]:
print(FINAL_MODEL)

In [ ]:
predict_model(FINAL_MODEL);

In [ ]:
unseen_predictions = predict_model(FINAL_MODEL, data=data_unseen)
unseen_predictions.head()

In [ ]:
plot_model(FINAL_MODEL, plot = 'auc')

In [ ]:
pkl_name = (today + "_" + model + "_" + pair)

In [ ]:
save_model(FINAL_MODEL, pkl_name)

In [ ]:
SAVED_FINAL_MODEL = load_model(pkl_name)

In [ ]:
new_prediction = predict_model(SAVED_FINAL_MODEL, data=data_unseen)

In [ ]:
data_unseen.shape

In [ ]:
new_prediction.head(20)

In [ ]:
file = open(today + "_" + model + "_binary_classification_log_" + pair + ".txt", "w")
file.write ("Date: " + today + "\n" + \
            "Currency Pair: " + pair + "\n" + \
            "Model: " + model + "\n" + \
            "Frac: " + str(frac) + "\n" + \
            "Random State: " + str(random_state) + "\n" )
file.close()